In [2]:
import pandas as pd
import xgboost as xgb
import datetime
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import pickle

Endpoint 1: Predecir el vehículo según el destino

Descripción: Este endpoint recibirá datos sobre el destino del viaje y devolverá una predicción del tipo de vehículo más adecuado para el mismo, considerando factores como la distancia, la disponibilidad de vehículos eléctricos y la eficiencia energética.
Parámetros de Entrada: Características del destino del viaje (por ejemplo, ubicación, distancia, hora del día).
Respuesta: Tipo de vehículo recomendado para el viaje

Endpoint 2: Predecir la demanda según día/noche y factores climáticos

Descripción: Este endpoint recibirá datos sobre el momento del día y las condiciones climáticas y devolverá una estimación de la demanda de transporte en función de estos factores, lo que permitirá una mejor asignación de recursos y una gestión más eficiente del servicio de transporte.

Parámetros de Entrada: Momento del día (día/noche), condiciones climáticas (temperatura, precipitación, etc.).
Respuesta: Estimación de la demanda de transporte para el período especificado.


In [3]:
modelos = pd.read_parquet('E:/Pruebas/Datasets_models/approvedModels.parquet')

In [4]:


# Reemplazar los valores en la columna 'tipo_color'
modelos['tipo_color'] = modelos['tipo_color'].replace({'Yellow': 'yellow', 'Green': 'green'})

# Mostrar el DataFrame resultante
modelos.head()


,modelsId,models,2017,2018,2019,2020,2021,2022,2023,tipo_color,type,Pax
0,1,Chevrolet Impala,1,1,0,0,0,0,0,green,Sedan,4
1,2,Chevrolet Malibu,1,1,1,1,0,0,0,green,Sedan,4
2,3,Chevrolet Malibu Hybrid,1,1,1,0,0,0,0,green,Sedan,4
3,4,Chrysler Pacifica Hybrid Accesible,1,1,1,1,0,0,0,yellow,Mini-Van,6
4,5,"Chrysler Pacifica Accesible, Rear entry (FR Co...",1,1,1,1,1,1,1,yellow,Mini-Van,5


In [6]:
modelos.to_parquet('E:/Pruebas/Datasets_models/approvedModels.parquet')

c:\Users\Potato\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [10]:
zonas = pd.read_parquet('Datasets_models\zones.parquet')
zonas

,ubicacion_inicio,barrio,zona,tipo_color
0,1,EWR,Newark Airport,Yellow
1,2,Queens,Jamaica Bay,Green
2,3,Bronx,Allerton/Pelham Gardens,Green
3,4,Manhattan,Alphabet City,Yellow
4,5,Staten Island,Arden Heights,Green
...,...,...,...,...
260,261,Manhattan,World Trade Center,Yellow
261,262,Manhattan,Yorkville East,Yellow
262,263,Manhattan,Yorkville West,Yellow
263,264,Unknown,NV,None


In [3]:
pip install google-auth


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
#conexion a bigquery para endpoints
import pandas_gbq
from google.oauth2 import service_account

# Ruta al archivo JSON de credenciales
credentials_path = 'E:/Pruebas/norse-journey-420017-a7ce9f7cb582.json'

# Cargar las credenciales desde el archivo JSON
credentials = service_account.Credentials.from_service_account_file(credentials_path)

# Definir la consulta a ejecutar en BigQuery
query = """
  SELECT 
  gy.anio,
  gy.mes,
  gy.dia_inicio,
  gy.hora_inicio,
  gy.hora_fin,
  gy.distancia_viaje,
  gy.ubicacion_inicio,
  gy.ubicacion_fin,
  gy.total_tiempo,
  pc.tipo_color,
  z.barrio,
  c.rain_sum as Lluvia,
  c.snowfall_sum as Nieve,
  c.temperature_2m_max as Temp_Maxima, 
  c.temperature_2m_min as Temp_Minima,
  m.models as modelos,
  m.type as tipo,
  m.Pax as pax

FROM norse-journey-420017.proyecto_taxis.taxis_completo AS gy
JOIN norse-journey-420017.proyecto_taxis.zonas AS z ON gy.ubicacion_inicio = z.ubicacion_inicio
JOIN norse-journey-420017.proyecto_taxis.puente_color AS pc ON gy.tipo_color = pc.tipo_color
JOIN norse-journey-420017.proyecto_taxis.climas AS c ON z.barrio = c.zone
JOIN norse-journey-420017.proyecto_taxis.modelos_aprobados AS m ON pc.tipo_color = m.tipo_color

WHERE gy.anio = 2023
ORDER BY RAND()
LIMIT 900000;
"""

# Lee directamente desde BigQuery a un DataFrame de pandas
df99 = pandas_gbq.read_gbq(query, project_id='norse-journey-420017', credentials=credentials)

# Ahora puedes imprimir o trabajar con el DataFrame df según necesites
df99.head()

Downloading: 100%|██████████|


,anio,mes,dia_inicio,hora_inicio,hora_fin,distancia_viaje,ubicacion_inicio,ubicacion_fin,total_tiempo,tipo_color,barrio,Lluvia,Nieve,Temp_Maxima,Temp_Minima,modelos,tipo,pax
0,2023,2,10,08:59:56,09:54:18,18.90,132,68,00:54:22,yellow,Queens,34.899998,0.0,12.556500,8.606501,Toyota Camry,Sedan,4
1,2023,12,16,01:32:27,01:53:31,5.93,79,238,00:21:04,yellow,Manhattan,0.000000,0.0,-3.173000,-9.973000,Toyota Highlander,SUV,4
2,2023,7,24,14:45:30,14:58:01,2.17,161,236,00:12:31,yellow,Manhattan,1.300000,0.0,22.476999,14.977000,Toyota Avalon Hybrid,Sedan,4
3,2023,11,2,23:06:48,23:46:09,10.61,142,228,00:39:21,yellow,Manhattan,0.000000,0.0,12.027000,3.877000,Chrysler Pacifica Hybrid Accesible,Mini-Van,6
4,2023,10,2,20:55:44,21:13:40,10.72,138,16,00:17:56,yellow,Queens,0.000000,0.0,25.956501,17.756500,Tesla Model 3 Long Range (EV),Sedan,4


In [4]:
df99.to_pickle('df_modelo_tv.pkl')

In [4]:
#definimos NUM_CLASSES segun el numero de clases en el problema de clasificación
NUM_CLASSES = len(df99['modelos'].unique())
#separamos las caracteristicas (features) y la variable objetivo (target)
x = df99.drop(['modelos', 'hora_inicio','hora_fin','total_tiempo','tipo_color','barrio','tipo','Lluvia','Nieve','Temp_Minima','Temp_Maxima'], axis=1) #características
y = df99['modelos'] #target

#dividimos los datos en conjuntos de entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)

#codificamos las etiquetas de  clase en números enteros
cod_num = LabelEncoder()
y_train_encoded = cod_num.fit_transform(y_train)

# cramos el clasificador XGBOOST
clf = xgb.XGBClassifier(objective='multi:softmax', num_class=NUM_CLASSES, random_state=42)

#entrenamos el modelo
clf.fit(x_train,y_train_encoded)

#hacemos predicciones en el conjunto de prueba
y_pred_encoded = clf.predict(x_test)

y_pred = cod_num.inverse_transform(y_pred_encoded)

#calculamos la precision del modelo
precision = accuracy_score(y_test, y_pred)

print('Precisión del modelo', precision)

Precisión del modelo 0.11015555555555556


In [6]:
#cargamos el modelo entrenado desde el archivo 
with open('df_modelo_tv.pkl', 'rb') as f:
    df99 = pickle.load(f)


In [7]:
#este funciona
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import xgboost as xgb

# Extraer solo la hora de las columnas hora_inicio y hora_fin
df99['hora_inicio'] = df99['hora_inicio'].apply(lambda x: x.hour)
df99['hora_fin'] = df99['hora_fin'].apply(lambda x: x.hour)

#df99['total_tiempo'] = pd.to_timedelta(df99['total_tiempo'])  # Convertir a tipo timedelta 
# Convertir el formato de total_tiempo a minutps
#df99['total_tiempo_minutos'] = df99['total_tiempo'].dt.total_seconds() / 60

# Eliminar la columna total_tiempo original
#df99.drop('total_tiempo', axis=1, inplace=True)


# Definir el número de clases
NUM_CLASSES = len(df99['modelos'].unique())

# Separar las características (features) y la variable objetivo (target)
x = df99.drop(['anio', 'total_tiempo','modelos','tipo_color','barrio','tipo','Lluvia','Nieve','Temp_Minima','Temp_Maxima'], axis=1) # características
y = df99['modelos'] # target

# Dividir los datos en conjuntos de entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Codificar las etiquetas de clase en números enteros
cod_num = LabelEncoder()
y_train_encoded = cod_num.fit_transform(y_train)

# Crear el clasificador XGBoost
clf = xgb.XGBClassifier(objective='multi:softmax', num_class=NUM_CLASSES, random_state=42)

# Entrenar el modelo
clf.fit(x_train, y_train_encoded)

# Hacer predicciones en el conjunto de prueba
y_pred_encoded = clf.predict(x_test)
y_pred = cod_num.inverse_transform(y_pred_encoded)

# Calcular la precisión del modelo
precision = accuracy_score(y_test, y_pred)

print('Precisión del modelo', precision)


Precisión del modelo 0.10942777777777778


In [8]:
print(x_train)

        mes  dia_inicio  hora_inicio  hora_fin  distancia_viaje  \
394657    3          31            8         8             1.80   
517437    2          23           15        15             2.21   
208118   11          15            9        10            17.70   
626437    7          29            1         2             0.74   
699970    2          15            7         7             0.57   
...     ...         ...          ...       ...              ...   
259178   12          23           23        23            16.59   
365838    7          10            2         2             4.55   
131932   10          28            0         0             1.23   
671155    5          29            0         1             2.79   
121958    6           5           19        19             1.65   

        ubicacion_inicio  ubicacion_fin  pax  
394657               162            140    4  
517437               164            125    4  
208118                75            132    4  
626437 

In [9]:
print(x_test)

        mes  dia_inicio  hora_inicio  hora_fin  distancia_viaje  \
527984    2          15           13        14             1.65   
618093    7          29           17        17             2.89   
484217   12           4           20        20             9.30   
161123    7          22           15        15             0.03   
718642    5          25           16        16             2.50   
...     ...         ...          ...       ...              ...   
28258     1          14            9        10             2.60   
71230    11          11           19        20             4.96   
504248   11           2           10        10             6.30   
284058    7          12           20        20             0.60   
518166   10          16           13        14            10.56   

        ubicacion_inicio  ubicacion_fin  pax  
527984               229            236    4  
618093               142            234    4  
484217               132            177    4  
161123 

In [10]:
print(x)

        mes  dia_inicio  hora_inicio  hora_fin  distancia_viaje  \
0         2          10            8         9            18.90   
1        12          16            1         1             5.93   
2         7          24           14        14             2.17   
3        11           2           23        23            10.61   
4        10           2           20        21            10.72   
...     ...         ...          ...       ...              ...   
899995   10          11           12        12             1.87   
899996   12           6           10        10             3.09   
899997   12           6           11        11             0.73   
899998    6          11           19        20            17.38   
899999   12          15           20        20             0.59   

        ubicacion_inicio  ubicacion_fin  pax  
0                    132             68    4  
1                     79            238    4  
2                    161            236    4  
3      

In [3]:

#cargamos el modelo entrenado desde el archivo 
with open('x_train.pkl', 'rb') as f:
    modelo_entrenado = pickle.load(f)

In [11]:

def predecir_vehiculo( mes, dia_inicio, hora_inicio, hora_fin,distancia_viaje, ubicacion_inicio, ubicacion_fin,pax, modelo_entrenado):
    # Crear un array numpy con las características
    datos = np.array([[ mes, dia_inicio, hora_inicio, hora_fin, distancia_viaje, ubicacion_inicio, ubicacion_fin, pax]])
    
    
    # Realizar la predicción usando el modelo
    prediccion = modelo_entrenado.predict(datos)
    
    # Decodificar la etiqueta predicha si es necesario
    tipo_vehiculo = cod_num.inverse_transform(prediccion)
    
    return tipo_vehiculo

mmodelo_entrenado = clf

count = 0
# Ejemplo de uso:
for indice, fila in x_train.iterrows():
    tipo_vehiculo_recomendado = predecir_vehiculo(fila['mes'], fila['dia_inicio'],fila['hora_inicio'], fila['hora_fin'],fila['distancia_viaje'], fila['ubicacion_inicio'], fila['ubicacion_fin'],fila['pax'], modelo_entrenado)
    print(f'Para el viaje {indice + 1}, se recomienda el vehiculo: {tipo_vehiculo_recomendado}')

    count +=1 
    if count == 100:
        break

Para el viaje 394658, se recomienda el vehiculo: ['Toyota Highlander Hybrid']
Para el viaje 517438, se recomienda el vehiculo: ['Tesla Model 3 Long Range (EV)']
Para el viaje 208119, se recomienda el vehiculo: ['Kia Niro Plug-In Hybrid']
Para el viaje 626438, se recomienda el vehiculo: ['Toyota Venza Hybrid']
Para el viaje 699971, se recomienda el vehiculo: ['Toyota Sienna Accessible, Rear Entry']
Para el viaje 691356, se recomienda el vehiculo: ['Toyota Camry Hybrid']
Para el viaje 122755, se recomienda el vehiculo: ['Hyundai Ionic 5 (EV)']
Para el viaje 704194, se recomienda el vehiculo: ['Toyota Camry Hybrid']
Para el viaje 721666, se recomienda el vehiculo: ['Kia Niro (EV)']
Para el viaje 8677, se recomienda el vehiculo: ['Ford Escape Hybrid']
Para el viaje 743256, se recomienda el vehiculo: ['Toyota Highlander Hybrid']
Para el viaje 146779, se recomienda el vehiculo: ['Kia Niro (EV)']
Para el viaje 109747, se recomienda el vehiculo: ['Kia Niro Hybrid']
Para el viaje 293974, se rec

In [14]:
x_train.to_pickle('xtrain.pkl')

In [6]:
import pickle

#guardamos el modelo entrenado en un archivo pkl
with open('modelo_entrenado.pkl', 'wb') as f:
    pickle.dump(clf,f)

In [21]:
#definimos NUM_CLASSES segun el numero de clases en el problema de clasificación
NUM_CLASSES = len(df99['modelos'].unique())
#separamos las caracteristicas (features) y la variable objetivo (target)
x = df99.drop(['modelos', 'hora_inicio','hora_fin','total_tiempo','tipo_color','barrio','tipo'], axis=1) #características
y = df99['modelos'] #target

#codificamos las etiquetas de clase en números enteros
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

#dividimos los datos en conjuntos de entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(x, y_encoded, test_size=0.2, random_state=42)

# Codificar las características categóricas en todo el conjunto de datos (entrenamiento y prueba)
x_train['ubicacion_inicio'] = label_encoder.fit_transform(x_train['ubicacion_inicio'])
x_train['ubicacion_fin'] = label_encoder.transform(x_train['ubicacion_fin'])
x_test['ubicacion_inicio'] = label_encoder.fit_transform(x_test['ubicacion_inicio'])
x_test['ubicacion_fin'] = label_encoder.transform(x_test['ubicacion_fin'])

# cramos el clasificador XGBOOST
clf = xgb.XGBClassifier(objective='multi:softmax', num_class=NUM_CLASSES, random_state=42)

#entrenamos el modelo
clf.fit(x_train, y_train)

#hacemos predicciones en el conjunto de prueba
y_pred = clf.predict(x_test)

#calculamos la precision del modelo
precision = accuracy_score(y_test, y_pred)

print('Precisión del modelo:', precision)


ValueError: y contains previously unseen labels: [48.0, 161.0, 239.0, 249.0]

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder


In [ ]:
etiquetas = data['etiquetas'].values